# Connecting to the Predictit 'API'

https://predictit.freshdesk.com/support/solutions/articles/12000001878-does-predictit-make-market-data-available-via-an-api-

In [1]:
import requests
import sys
import configparser
import datetime
import pytz

import pandas as pd
import tweepy

from bs4 import BeautifulSoup

sys.path.append("..")
from src.tweetdownloader import TweetDownloader
import src.utils as utils
import src.poisson as poisson

In [2]:
# we'll start with rdt
market_number = '6689'
url = f"https://www.predictit.org/api/marketdata/markets/{market_number}/"
url

'https://www.predictit.org/api/marketdata/markets/6689/'

In [3]:
response = requests.get(url)

In [4]:
response_data = response.json()

In [5]:
response_data

{'id': 6689,
 'name': 'How many tweets will @realDonaldTrump post from noon May 6 to 13?',
 'shortName': '@realDonaldTrump tweets 5/6 to 5/13?',
 'image': 'https://az620379.vo.msecnd.net/images/Markets/36231f71-30ba-47c1-8f5d-c7481e014c11.jpg',
 'url': 'https://www.predictit.org/markets/detail/6689/How-many-tweets-will-@realDonaldTrump-post-from-noon-May-6-to-13',
 'contracts': [{'id': 22581,
   'dateEnd': '2020-05-13T12:00:00',
   'image': 'https://az620379.vo.msecnd.net/images/Contracts/small_36231f71-30ba-47c1-8f5d-c7481e014c11.jpg',
   'name': '240 or more',
   'shortName': '240+',
   'status': 'Open',
   'lastTradePrice': 0.47,
   'bestBuyYesCost': 0.49,
   'bestBuyNoCost': 0.53,
   'bestSellYesCost': 0.47,
   'bestSellNoCost': 0.51,
   'lastClosePrice': 0.4,
   'displayOrder': 8},
  {'id': 22586,
   'dateEnd': '2020-05-13T12:00:00',
   'image': 'https://az620379.vo.msecnd.net/images/Contracts/small_36231f71-30ba-47c1-8f5d-c7481e014c11.jpg',
   'name': '220 - 229',
   'shortName':

## Construct buckets with their costs

In [6]:
title = response_data["contracts"][0]["name"]

In [7]:
def num_from_title(title): return [int(i) for i in title.split() if i.isdigit()]

In [8]:
lower = min([num_from_title(i["name"])[0] for i in response_data["contracts"]])

In [9]:
title = response_data["contracts"][1]["name"]
gaps = num_from_title(title)
gap = gaps[1] - gaps[0] + 1

## get the number of tweets since the start

yeah we're gonig to need to change this

In [10]:
end_date = pd.to_datetime(response_data["contracts"][0]["dateEnd"])
end_date = end_date.tz_localize("US/Eastern")


In [11]:
start_date = end_date - datetime.timedelta(days=7)
start_date

Timestamp('2020-05-06 12:00:00-0400', tz='US/Eastern')

In [12]:
time_left = end_date - datetime.datetime.now(tz=pytz.timezone('US/Eastern'))

In [13]:
datetime.datetime.now(tz=pytz.timezone('US/Eastern'))

datetime.datetime(2020, 5, 11, 6, 36, 55, 566040, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)

In [14]:
time_left.days

2

In [15]:
days_left = time_left.days + round((time_left.seconds / 2880) - (time_left.days * 2) / 24) * .01

In [16]:
rdt_tweets = TweetDownloader(to_load="realDonaldTrump")

INFO:root:API created
INFO:root:Liberating realDonaldTrump
INFO:root:	Page: 0
INFO:root:Found: 1 duplicate rows
INFO:root:Writing ../data/realDonaldTrump.csv


In [17]:
tweets = rdt_tweets.dataset

In [18]:
tweets_so_far = len(tweets[:start_date])

### TODO from here

Now we need to:

* [ ] make a bucket dit for the prices
* [ ] make kelly calcs

In [19]:
def process_name(name):
    numbers = num_from_title(name)
    if 'fewer' in name:
        return str(numbers[0])
    elif 'more' in name:
        return str(numbers[0])
    else:
        return str(numbers[0]) + '-' + str(numbers[1])

In [20]:
prices = dict()
for contract in response_data["contracts"]:
    prices[process_name(contract["name"])] = contract["bestBuyYesCost"]

In [21]:
prices

{'240': 0.49,
 '220-229': 0.19,
 '230-239': 0.19,
 '210-219': 0.12,
 '200-209': 0.08,
 '190-199': 0.04,
 '180-189': 0.01,
 '170-179': 0.01,
 '169': 0.01}

In [30]:
def setup_data(name, days=90):
    df = utils.tweets(name)
    by_day = df.groupby(df.index.date).count()
    print(by_day)
    cutoff_date = by_day.iloc[len(by_day)-1].name - datetime.timedelta(days=days)
    return by_day[cutoff_date:]

In [31]:
odds = poisson.basic_poisson(setup_data('realDonaldTrump')["ID"], days_left, tweets_so_far, lower=lower, gap=gap)

             ID
2017-01-01    6
2017-01-02    7
2017-01-03   10
2017-01-04   10
2017-01-05    6
...         ...
2020-05-07   10
2020-05-08   12
2020-05-09   22
2020-05-10  109
2020-05-11   21

[1214 rows x 1 columns]
29.032967032967033
2.07
60.098241758241755
60.0


In [24]:
odds

{'169': 0.0,
 '170-179': 0.0,
 '180-189': 0.0,
 '190-199': 0.0,
 '200-209': 0.0,
 '210-219': 0.8999999999999999,
 '220-229': 15.299999999999999,
 '230-239': 43.1,
 '240': 32.0}

In [25]:
tweets_so_far

177

In [26]:
def kelly(odds, prices):
    df = pd.DataFrame([odds,prices])
    df = df.T
    df.columns = ["odds", "prices"]
    df["odds"] = df["odds"] * .01
    df["win"] = 1 - df["prices"]
    df["b"] = df["win"] / df["prices"]
    df["q"] = 1 - df["odds"]
    df["f"] = df["odds"] - df["q"] / df["b"]
    return df
    
t = kelly(odds, prices)

In [27]:
t

,odds,prices,win,b,q,f
169,0.000,0.01,0.99,99.000000,1.000,-0.010101
170-179,0.000,0.01,0.99,99.000000,1.000,-0.010101
180-189,0.000,0.01,0.99,99.000000,1.000,-0.010101
190-199,0.000,0.04,0.96,24.000000,1.000,-0.041667
200-209,0.000,0.08,0.92,11.500000,1.000,-0.086957
210-219,0.009,0.12,0.88,7.333333,0.991,-0.126136
220-229,0.153,0.19,0.81,4.263158,0.847,-0.045679
230-239,0.431,0.19,0.81,4.263158,0.569,0.297531
240,0.320,0.49,0.51,1.040816,0.680,-0.333333


## Potus

In [28]:
# download tweets before this, add this to pipeline

# iterate through this?
market_numbers = {"whitehouse":6692,
                 "potus":6696,
                 "realDonaldTrump":6689}

In [29]:
def response_data{}

SyntaxError: invalid syntax (<ipython-input-29-f6bb4bc96c88>, line 1)